In [1]:
import sys
sys.path.append('..')

from src.tool.tool_registry import ToolRegistry
from langchain_core.messages import SystemMessage, AIMessage, HumanMessage
from langchain_openai import ChatOpenAI
from langchain.agents import create_agent

model = ChatOpenAI(
    model="qwen3-8B", 
    base_url="http://127.0.0.1:1234/v1", 
    api_key=""
)

tool_registry = ToolRegistry()
tools = tool_registry.get_tools()

agent_executor = create_agent(model, tools)


In [2]:
chat_history = [
    SystemMessage(
    """
        You are a expert data analyst. Use the provided tools to answer user questions based on the graph database schema and data.
        Registered Tool:
        1. schema_linking: Provides the graph database schema definition to help understand the data structure.
        2. execute_cypher: Executes Cypher queries against the graph database and returns results.

        When given a user question, first use the schema_linking tool to get the database schema. 
        Then, formulate an appropriate Cypher query to retrieve the relevant data using the execute_cypher tool. 
        Check the result, it's fraud or not. Give your confidence level (high, medium, low) on your answer.

        INSTRUCTIONS:
        1. Search the database for the RULES regarding Diagnosis from the input. 
        (Look for relationships like REQUIRES or TYPICALLY_TREATED_WITH).
        2. Compare the 'Procedures Performed' in this claim against the required procedures found in the database.
        3. VERDICT: Is this claim compliant or suspicious? Explain why based on the missing evidence.
    """,
    ),
    HumanMessage("See the valid claim examples from the graph database schema first."),
    AIMessage("Okay, I will first retrieve the graph database schema using the schema_linking tool. And then I will formulate a Cypher query to get the valid claim examples."),
    HumanMessage("Compare the input claim with the valid claim examples and determine if it is fraudulent."),
    AIMessage("Understood, I will analyze the input claim against the valid claim examples from the graph database to assess its authenticity.")
]

In [3]:
from langchain_core.callbacks import BaseCallbackHandler

# --- 2. Define the Custom Printer Class ---
class ToolExecutionPrinter(BaseCallbackHandler):
    def on_tool_start(self, serialized, input_str, **kwargs):
        """Run when a tool starts running."""
        tool_name = serialized.get("name")
        print(f"\n[LOG] 🛠️  Agent is entering tool: {tool_name}")
        print(f"[LOG]    Input args: {input_str}\n")

    def on_tool_end(self, output, **kwargs):
        """Run when a tool ends running."""
        print(f"[LOG] ✅ Tool execution finished.\n")

In [4]:
user_input = """
I have a NEW claim that I want to validate for potential fraud. 
Compare this claim data against the 'Medical Ontology' rules in the database.

CLAIM DETAILS:
- Diagnosis Code: I21.9 (Acute Myocardial Infarction)
- Procedures Performed: ["Oxygen Therapy", "Observation"]
- Clinical Notes: "Patient came with chest pain. Given oxygen. Condition stable."
- Total Cost: 15,000,000 IDR
"""

final_messages = chat_history + [HumanMessage(content=user_input)]

response = agent_executor.invoke({
    "messages": final_messages,
},
config={
        "callbacks": [ToolExecutionPrinter()]
    }
)

print(f"Response: {response['messages'][-1].content}")


[LOG] 🛠️  Agent is entering tool: schema_linking
[LOG]    Input args: {'question': 'What are the valid claim examples and their associated diagnosis rules in the Medical Ontology?'}

[LOG] ✅ Tool execution finished.


[LOG] 🛠️  Agent is entering tool: execute_cypher
[LOG]    Input args: {'cypher_query': "MATCH (d:Diagnosis {code: 'I21.9'})-[:REQUIRES]->(p:Procedure) RETURN p.name AS required_procedure"}

[LOG] ✅ Tool execution finished.


[LOG] 🛠️  Agent is entering tool: execute_cypher
[LOG]    Input args: {'cypher_query': "MATCH (d:Diagnosis {code: 'I21.9'})-[:TYPICALLY_TREATED_WITH]->(p:Procedure) RETURN p.name AS typical_procedure"}

[LOG] ✅ Tool execution finished.


[LOG] 🛠️  Agent is entering tool: execute_cypher
[LOG]    Input args: {'cypher_query': "MATCH (c:Claim)-[:CODED_AS]->(d:Diagnosis {code: 'I21.9'}) RETURN c.total_cost AS claim_total, d.avg_cost AS diagnosis_avg_cost"}

[LOG] ✅ Tool execution finished.

Response: <think>
Okay, let's break this down. The user's claim h

In [ ]:
system_prompt = """
You are an expert Medical Fraud Detection AI Agent powered by a Knowledge Graph.
Your goal is to validate insurance claims against medical rules.

STRATEGY (Graph RAG Process):
1. **Understand the Goal**: Analyze the user's question to identify key entities (e.g., Claim IDs, Diagnoses).
2. **Check Schema**: ALWAYS call the `schema_linking` tool first to understand the database structure.
3. **Formulate Query**: Based on the schema, write a syntactically correct Cypher query to retrieve the necessary facts.
   - Example: To check fraud, you often need to compare `(:Claim)-[:CODED_AS]->(:Diagnosis)` 
     with the actual `(:Claim)-[:HAS_CLINICAL_NOTE]...` evidence.
4. **Execute**: Run the query using `execute_cypher`.
5. **Reason**: Compare the structured rules (e.g., Diagnosis X REQUIRES Procedure Y) against the claim evidence.
6. **Answer**: Provide a verdict based ONLY on the data retrieved.

CONSTRAINTS:
- Do not invent medical rules; query them from the database relationships (REQUIRES, TYPICALLY_TREATED_WITH).
- If a procedure is missing from the clinical notes but required by the diagnosis, flag it as SUSPICIOUS.
"""